In [1]:
import cv2
import itertools, os, time
import numpy as np
from Model import get_Model
from parameter import letters
import argparse
from keras import backend as K
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
K.set_learning_phase(0)

Region = {"A": "서울 ", "B": "경기 ", "C": "인천 ", "D": "강원 ", "E": "충남 ", "F": "대전 ",
          "G": "충북 ", "H": "부산 ", "I": "울산 ", "J": "대구 ", "K": "경북 ", "L": "경남 ",
          "M": "전남 ", "N": "광주 ", "O": "전북 ", "P": "제주 "}
Hangul = {"dk": "아", "dj": "어", "dh": "오", "dn": "우", "qk": "바", "qj": "버", "qh": "보", "qn": "부",
          "ek": "다", "ej": "더", "eh": "도", "en": "두", "rk": "가", "rj": "거", "rh": "고", "rn": "구",
          "wk": "자", "wj": "저", "wh": "조", "wn": "주", "ak": "마", "aj": "머", "ah": "모", "an": "무",
          "sk": "나", "sj": "너", "sh": "노", "sn": "누", "fk": "라", "fj": "러", "fh": "로", "fn": "루",
          "tk": "사", "tj": "서", "th": "소", "tn": "수", "gj": "허"}

def decode_label(out):
    # out : (1, 32, 42)
    out_best = list(np.argmax(out[0, 2:], axis=1))  # get max index -> len = 32
    out_best = [k for k, g in itertools.groupby(out_best)]  # remove overlap value
    outstr = ''
    for i in out_best:
        if i < len(letters):
            outstr += letters[i]
            '''
            if letters[i] != "X":
                outstr += letters[i]
            '''
    return outstr


def label_to_hangul(label):  # eng -> hangul
    #print("label in prediction:",label)
    
    if len(label)==10 and label[9] == "X":
        region = label[0]
        two_num = label[1:3]
        hangul = label[3:5]
        four_num = label[5:9]
        try:
            region = Region[region] if region != 'Z' else ''
        except:
            pass
        try:
            hangul = Hangul[hangul]
        except:
            pass
        return region + two_num + hangul + four_num
    else:
        #print("label:",label)
        region = label[0]
        three_num = label[1:4]
        hangul = label[4:6]
        four_num = label[6:]
        if len(four_num)==5:
            four_num = label[6:10]
        try:
            region = Region[region] if region != 'Z' else ''
        except:
            pass
        try:
            hangul = Hangul[hangul]
        except:
            pass
        return region + three_num + hangul + four_num

#parser = argparse.ArgumentParser()
#parser.add_argument("-w", "--weight", help="weight file directory",
#                    type=str, default="LSTM+BN5--30--0.001.hdf5")
'''
parser.add_argument("-t", "--test_img", help="Test image directory",
                    type=str, default="./DB/real_test/")
'''
#args = parser.parse_args()

# Get CRNN model
model = get_Model(training=False)

try:
    model.load_weights("LSTM+BN5--30--0.001.hdf5")
    print("...Previous weight data...")
except:
    raise Exception("No weight file!")


#test_dir =args.test_img
#test_imgs = os.listdir(args.test_img)
total = 0
acc = 0
letter_total = 0
letter_acc = 0
start = time.time()
import pdb
'''
for test_img in test_imgs:
    img = cv2.imread(test_dir + test_img, cv2.IMREAD_GRAYSCALE)
    ret, img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY) 
    #print(type(img))
    #plt.imshow(img)
    #pdb.set_trace()

    img_pred = img.astype(np.float32)
    img_pred = cv2.resize(img_pred, (128, 64))
    img_pred = (img_pred / 255.0) * 2.0 - 1.0
    img_pred = img_pred.T
    img_pred = np.expand_dims(img_pred, axis=-1)
    img_pred = np.expand_dims(img_pred, axis=0)
    ret, thr1 = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY) 
    ret, thr2 = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV) 

    net_out_value = model.predict(img_pred)

    pred_texts = decode_label(net_out_value)

    for i in range(min(len(pred_texts), len(test_img[0:-4]))):
        if pred_texts[i] == test_img[i]:
            letter_acc += 1
    letter_total += max(len(pred_texts), len(test_img[0:-4]))

    if pred_texts == test_img[0:-4]:
        acc += 1
    print("pred_texts:",pred_texts)
    #print("test_img:",test_img[0:-4])
    total += 1
    print('Predicted: %s  /  True: %s' % (label_to_hangul(pred_texts), label_to_hangul(test_img[0:-4])))
    
    cv2.rectangle(img, (0,0), (150, 30), (0,0,0), -1)
    cv2.putText(img, pred_texts, (5, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255),2)

    cv2.imshow("q", img)
    if cv2.waitKey(0) == 27:
       break
    cv2.destroyAllWindows()

'''
cap = cv2.VideoCapture(0)
while(cap.isOpened()):
    #img = cv2.imread(test_dir + test_img, cv2.IMREAD_GRAYSCALE)
    ret, frame = cap.read()
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH) 
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    #frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    cv2.imshow("test",frame)
    #print(type(img))
    #plt.imshow(img)
    #pdb.set_trace()

    img_pred = frame.astype(np.float32)
    img_pred = cv2.resize(img_pred, (128, 64))
    img_pred = (img_pred / 255.0) * 2.0 - 1.0
    img_pred = img_pred.T
    img_pred = np.expand_dims(img_pred, axis=-1)
    img_pred = np.expand_dims(img_pred, axis=0)

    '''
    ret, thr1 = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY) 
    ret, thr2 = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV) 
    '''

    net_out_value = model.predict(img_pred)

    pred_texts = decode_label(net_out_value)
    print("pred_texts:",pred_texts)
    print("width:" ,width)
    print("height:", height)
    #print("test_img:",test_img[0:-4])
    #total += 1
    #print('Predicted: %s  /  True: %s' % (label_to_hangul(pred_texts), label_to_hangul(test_img[0:-4])))
    cv2.rectangle(frame, (0,0), (150, 30), (0,0,0), -1)
    cv2.putText(frame, pred_texts, (5, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255),2)
    cv2.imshow("q", frame)
    if cv2.waitKey(60) > 0: break
        
cv2.release()
cv2.destroyAllWindows()

    



Using TensorFlow backend.









Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.cast` instead.
...Previous weight data...


error: OpenCV(4.3.0) C:\projects\opencv-python\opencv\modules\highgui\src\window.cpp:376: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'
